In [12]:
from psycopg2 import connect as cnct
from datetime import datetime
from scipy.stats import mannwhitneyu
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# 0. Подготовка глобальных переменных

In [2]:
DB_NAME = 'orderlogs'
DB_SETTINGS = {
    'dbname': DB_NAME,
    'user': 'postgres',
    'password': '',
    'host': 'localhost',
    'port': 5432
}

with cnct(**DB_SETTINGS) as conn:
    cur = conn.cursor()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
    TABLE_NAMES = cur.fetchall()
    TABLE_NAMES = sorted(list(map(lambda x: x[0], TABLE_NAMES)))

SECCODES = sorted(list(pd.read_excel(
    os.path.join(
        os.path.abspath(''), 
        'data_results/selection/volume_instruments_specification.xlsx'))['Торговый код']))

PATH = os.path.join(os.path.abspath(''), './data_results/liquidity_time_series')

# 1. Сборка результатов моделирования по различным тикерам

In [3]:
def datetime_convertion(date, time):
    '''This function converts time in int format like 20190101, 125959000000 to POSIX
    
    :date: date in int format
    :time: time in int format
    '''
    year = int(str(date)[:4])
    month = int(str(date)[4:6])
    day = int(str(date)[6:])

    hours = int(str(time)[:2])
    minutes = int(str(time)[2:4])
    seconds = int(str(time)[4:6])
    microseconds = int(str(time)[6:])

    return datetime(year, month, day, hours, minutes, seconds, microseconds)

In [4]:
def table_results_combine(ticker):
    '''This function combines the results for one ticker over all days in the data base
    
    :ticker: the name of fin. instrument in the str format (for example, 'AFKS')
    :return: pd.DataFrame object with fields SECCODE, Date, TIME, Alpha_B, Alpha_S, Beta_B, Beta_S
    '''
    lst_files = list(filter(lambda x: ticker in x, os.listdir(PATH)))

    dataframe_ticker = pd.DataFrame(columns=['Date', 'TIME', 'SECCODE', 'Alpha_B', 'Beta_B', 'Alpha_S', 'Beta_S'])
    for path_to_file in lst_files:
        df = pd.read_csv(os.path.join(PATH, path_to_file), index_col=0)
        df['Date'] = int(path_to_file[-12:-4])
        df['SECCODE'] = path_to_file[:-13]
        dataframe_ticker = pd.concat((dataframe_ticker, df), sort=False)
    
    dataframe_ticker['DATETIME'] = np.array(list(map(lambda x: datetime_convertion(*x), zip(dataframe_ticker.Date, dataframe_ticker.TIME))))
    dataframe_ticker = dataframe_ticker[['SECCODE', 'DATETIME', 'Alpha_B', 
                                         'Alpha_S', 'Beta_B', 'Beta_S']].sort_values(by=['SECCODE', 'DATETIME']).reset_index(drop=True)
    
    return dataframe_ticker

In [5]:
def plot_results_tightness(dataframe, export=None):
    '''This function plots the results and saves into the .pdf
    
    :dataframe: pd.DataFrame object - the return of table_results_combine function
    '''
    ticker = dataframe.SECCODE[0]

    plt.figure()
    plt.plot(dataframe.DATETIME, dataframe.Alpha_B)
    plt.plot(dataframe.DATETIME, dataframe.Alpha_S)
    plt.legend(['Tightness Buy', 'Tightness Sell'])
    plt.title('The Visualization of Tightness for {}'.format(ticker))
    plt.xlabel('Date')
    plt.ylabel('Value of Parameter')
    plt.xticks(rotation=15)
    if export is None:
        plt.show()
    else:
        plt.savefig(os.path.join(os.path.abspath(''), 
                                 './data_results/liquidity_figures/{}_tightness.pdf'.format(ticker)),
                   bbox_inches='tight')
        plt.close()

In [6]:
def plot_results_depth(dataframe, export=None):
    '''This function plots the results and saves into the .pdf
    
    :dataframe: pd.DataFrame object - the return of table_results_combine function
    '''
    ticker = dataframe.SECCODE[0]

    plt.figure()
    plt.plot(dataframe.DATETIME, dataframe.Beta_B)
    plt.plot(dataframe.DATETIME, dataframe.Beta_S)
    plt.legend(['Depth Buy', 'Depth Sell'])
    plt.title('The Visualization of Depth for {}'.format(ticker))
    plt.xlabel('Date')
    plt.ylabel('Value of Parameter')
    plt.xticks(rotation=15)
    if export is None:
        plt.show()
    else:
        plt.savefig(os.path.join(os.path.abspath(''), 
                                 './data_results/liquidity_figures/{}_depth.pdf'.format(ticker)), 
                   bbox_inches='tight')
        plt.close()

In [7]:
df_over_all_tickers = pd.DataFrame(columns=['SECCODE', 'DATETIME', 'Alpha_B', 
                                            'Alpha_S', 'Beta_B', 'Beta_S'])
for seccode in SECCODES:
    print(seccode)
    df = table_results_combine(seccode)
    df_over_all_tickers = pd.concat((df_over_all_tickers, df), ignore_index=True)
    df.to_csv(os.path.join(os.path.abspath(''), 
                           f'./data_results/liquidity_time_series_aggregated/{seccode}_liquidity_parameters.csv'))
    plot_results_depth(df, export=True)
    plot_results_tightness(df, export=True)
df_over_all_tickers = df_over_all_tickers.reset_index(drop=True)
df_over_all_tickers.to_csv(os.path.join(os.path.abspath(''), 
                                        f'./data_results/liquidity_time_series_aggregated/ALL_SECCODES_liquidity_parameters.csv'))

AFKS


/usr/local/anaconda3/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


AFLT
ALRS
CBOM
CHMF
DSKY
ENRU
FEES
FIVE
GAZP
GMKN
HYDR
IRAO
LKOH
LNTA
LSRG
MAGN
MGNT
MOEX
MSNG
MTLR
MTLRP
MTSS
MVID
NLMK
NVTK
PHOR
PIKK
PLZL
POLY
ROSN
RSTI
RTKM
RUAL
SBER
SBERP
TATN
TATNP
TGKA
TRMK
TRNFP
UPRO
VTBR
YNDX
